In [1]:
import pandas as pd
import numpy as np
import sys
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import warnings
from IPython.display import clear_output
from multiprocessing import Pool
from time import time
warnings.filterwarnings('ignore')

In [2]:
def get_time_spaced_traindata(data):
    data['delta_t'] = np.int64((data.index - data.index[0]).total_seconds() // 3600) + 1
#     data = data.drop(columns=['station_id'])
#     data.index = range(len(data))
    return data

def get_time_spaced_testdata(data):
    data['delta_t'] = np.int64((data.index - data.index[0]).total_seconds() // 3600) + 1  
#     data = data.drop(columns=['station_id'])
#     data.index = range(len(data))
    aq_data = data['PM25_Concentration']
    data.drop(columns=['PM25_Concentration'])
    return data,aq_data

In [3]:
fold=1 # not using for loop to avoid ram overflow
print('fold', fold)

train_data = pd.read_csv('../data/processed/fold_'+str(fold)+'_train_mar_nsgp.csv.gz')
train_data['time'] = pd.to_datetime(train_data['time'])
train_data = train_data.set_index('time').sort_values(['time', 'station_id'])

test_data = pd.read_csv('../data/processed/fold_'+str(fold)+'_test_mar_nsgp.csv.gz')
test_data['time'] = pd.to_datetime(test_data['time'])
test_data = test_data.set_index('time').sort_values(['time', 'station_id'])

fold 1


In [4]:
train_input = get_time_spaced_traindata(train_data['2015-03-01':'2015-03-31'])
test_input,test_output = get_time_spaced_testdata(test_data['2015-03-01':'2015-03-31'])

In [5]:
display(train_input,test_input,test_output)

,station_id,longitude,latitude,PM25_Concentration,weather,temperature,humidity,wind_speed,wind_direction,filled,delta_t
time,,,,,,,,,,,
2015-03-01 00:00:00,1003,116.184239,39.914409,21.0,14.0,4.000000,17.0,8.50,24.0,False,1
2015-03-01 00:00:00,1005,116.136045,39.742767,25.0,1.0,2.000000,26.0,4.68,23.0,False,1
2015-03-01 00:00:00,1006,116.287451,39.987313,30.0,14.0,3.000000,23.0,3.00,14.0,False,1
2015-03-01 00:00:00,1008,116.348991,39.954047,68.0,14.0,3.000000,23.0,3.00,14.0,False,1
2015-03-01 00:00:00,1010,116.394009,39.876184,40.0,14.0,4.000000,18.0,3.00,24.0,False,1
...,...,...,...,...,...,...,...,...,...,...,...
2015-03-31 23:00:00,1027,116.628000,40.328000,83.0,1.0,8.000000,83.0,1.44,24.0,False,744
2015-03-31 23:00:00,1028,116.831999,40.369999,83.0,1.0,5.500000,86.5,1.26,14.0,False,744
2015-03-31 23:00:00,1030,116.220000,40.292000,103.0,1.0,9.666667,77.0,3.00,14.0,False,744


,station_id,longitude,latitude,PM25_Concentration,weather,temperature,humidity,wind_speed,wind_direction,filled,delta_t
time,,,,,,,,,,,
2015-03-01 00:00:00,1002,116.205310,40.003950,26.0,14.0,3.0,23.0,3.00,14.0,False,1
2015-03-01 00:00:00,1004,116.171150,39.815128,25.0,14.0,4.0,18.0,3.00,24.0,False,1
2015-03-01 00:00:00,1007,116.397400,39.982053,69.0,14.0,1.0,29.0,14.04,24.0,False,1
2015-03-01 00:00:00,1023,116.655000,40.127000,175.0,1.0,0.0,68.0,6.12,23.0,False,1
2015-03-01 00:00:00,1025,116.105999,39.936999,30.0,1.0,3.0,17.0,6.84,24.0,False,1
...,...,...,...,...,...,...,...,...,...,...,...
2015-03-31 23:00:00,1029,115.971999,40.453000,127.0,1.0,7.5,83.0,5.22,13.0,False,744
2015-03-31 23:00:00,1031,115.988000,40.365000,123.0,1.0,7.5,83.0,5.22,13.0,False,744
2015-03-31 23:00:00,1032,116.911000,40.499000,77.0,1.0,5.5,86.5,1.26,14.0,False,744


time
2015-03-01 00:00:00     26.0
2015-03-01 00:00:00     25.0
2015-03-01 00:00:00     69.0
2015-03-01 00:00:00    175.0
2015-03-01 00:00:00     30.0
                       ...  
2015-03-31 23:00:00    127.0
2015-03-31 23:00:00    123.0
2015-03-31 23:00:00     77.0
2015-03-31 23:00:00     62.0
2015-03-31 23:00:00     79.0
Name: PM25_Concentration, Length: 7440, dtype: float64

In [6]:
display(train_input.shape, test_input.shape, test_output.shape)

(14880, 11)

(7440, 11)

(7440,)

In [7]:
train_input.reset_index().to_csv('../data/time_feature/fold'+str(fold)+'/train_data_mar_nsgp.csv.gz', index=None)
test_input.reset_index().to_csv('../data/time_feature/fold'+str(fold)+'/test_data_mar_nsgp.csv.gz', index=None)
test_output.reset_index().to_csv('../data/time_feature/fold'+str(fold)+'/test_output_mar_nsgp.csv.gz', index=None)